In this notebook, different variable values can be tested programmatically, to increase a score.

In [ ]:
from tf.transformations import euler_from_quaternion, euler_matrix
import ros_numpy
import pcl
import time
import numpy as np
import math
import rosbag
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
bag_path = '/home/user/rosbags/big/hdl_slam_validation/2021-10-07-08-32-58.bag'
# Load rosbag
bag = rosbag.Bag(bag_path)

# Extract data from hdl bag
# Contains raw lidar data and the position/orientation of car
pose = []
for topic, msg, t in bag.read_messages(topics='/odom'):
    pose.append(msg.pose.pose)
lidar = []
for topic, msg, t in bag.read_messages(topics='/rslidar_points'):
    lidar.append(msg)

# To synchronize both topics, remove first of odom and the last 2 of rslidar
pose = pose[1:]
lidar = lidar[:-2]

In [ ]:
# My ROS Node
class VisualDetection():

    def __init__(self, ransac_distance_thresh=0.017, ransac_weight=0.125, ransac_make_seg=50,
    voxel_leaf=0.1, plane_near_ground=0.8, cut=[0, 30, -3.5, 3.5, -1, 1.5], max_iter=4,
    min_points=100):
        self.distance_threshold = ransac_distance_thresh
        self.distance_weight = ransac_weight
        self.ransac_make_segmenter_normals = ransac_make_seg
        self.plane_near_ground = plane_near_ground
        self.cut = cut
        self.min_points = min_points
        self.max_iterations = max_iter
        self.voxel_leaf = voxel_leaf

    def spin(self, cloud, pose):
        self.cloud = cloud
        self.pose = pose
        # Convert PointCloud2 msg to numpy array
        pc_array = ros_numpy.point_cloud2.pointcloud2_to_xyz_array(self.cloud, remove_nans=True)
        # Apply lidar to car frame transformation
        pc_array_tf = self.transform_pc(pc_array, roll=0, pitch=25.53, yaw=0)
        # Filter unwanted points (to reduce point cloud size) with passthrough filter
        pc_array_cut = self.reduce_pc(pc_array_tf, self.cut)
        # Convert numpy array to pcl object
        pc_cut = pcl.PointCloud()
        pc_cut.from_array(pc_array_cut.astype('float32'))
        # Downsample point cloud using voxel filter to further decrease size
        pc_small = self.voxel_filter(pc_cut, self.voxel_leaf)
        # Perform RANSAC until no new planes are being detected
        plane_coor, data = self.plane_detection(pc_small, self.min_points, self.max_iterations)
        return plane_coor, data

    def transform_pc(self, pc, roll=0, pitch=0, yaw=0, transl_x=1.753, transl_y=0, transl_z=1.156):
        """Transformation from Lidar frame to car frame. Rotation in rad and translation in m."""
        # Rotation matrix
        rot = euler_matrix(roll, pitch, yaw, 'sxyz')[:3, :3]
        # Apply rotation
        pc_tf = np.inner(pc, rot)
        # Translation
        translation = [transl_x, transl_y, transl_z]
        # Combine rotation and translation
        pc_tf += translation
        return pc_tf

    def reduce_pc(self, pc, cut):
        """Removes points outside of box"""
        x_lower, x_upper, y_lower, y_upper, z_lower, z_upper = cut
        pc_cut = pc[(pc[:, 0] > x_lower) & (pc[:, 0] < x_upper) & (pc[:, 1] > y_lower) & (
            pc[:, 1] < y_upper) & (pc[:, 2] > z_lower) & (pc[:, 2] < z_upper)]
        return pc_cut

    def voxel_filter(self, pc, leaf_size):
        """Downsample point cloud using voxel filter"""
        vg = pc.make_voxel_grid_filter()
        # Leaf_size is the length of the side of the voxel cube in m
        vg.set_leaf_size(leaf_size, leaf_size, leaf_size)
        pc_filtered = vg.filter()
        # print('Reduced size from {} to {}'.format(pc.size, pc_filtered.size))
        return pc_filtered

    def plane_detection(self, pc, min_points, max_planes):
        """Detects all planes in point cloud"""
        # Ground vector
        g_vec = None
        counter = 0
        while pc.size > min_points and counter < max_planes:
            # Detect most dominate plane and get inliers and normal vector
            indices, coefficients = self.ransac(pc)
            n_vec = coefficients[:-1]

            # Split pointcloud in inliers and outliers of plane
            pc, plane, pc_points = self.split_pc(pc, indices)

            # Exit if plane is empty
            if not plane:
                return [], []

            # Ignore walls to the side or in front
            if self.is_plane_near_ground(n_vec, self.plane_near_ground):
                # First ground like detection is most probably the ground
                if g_vec is None:
                    g_vec = n_vec
                # Either ground is detected again or potential ramp
                else:
                    is_ramp, data = self.ramp_detection(plane, g_vec, n_vec, 4, 7, 0, 5, 2, 4, 0, 10)
                    if is_ramp:
                        # Transform plane from local rslidar coordinates to global map coordinates
                        plane_global = self.relative_to_absolute(plane)
                        return plane_global, data
                    else:  
                        continue
            counter += 1
        return [], []

    def relative_to_absolute(self, pc):
        """Transforms relative lidar data to absolute by adding translation rotating"""
        # pc_arr = pc.to_array()
        pc_arr = np.array(pc)
        # Odometer
        pos = self.pose.position
        translation = [pos.x, pos.y, pos.z]
        ori = self.pose.orientation
        quat = [ori.x, ori.y, ori.z, ori.w]
        roll, pitch, yaw = euler_from_quaternion(quat)
        # Rotation matrix
        rot = euler_matrix(roll, pitch, yaw, 'sxyz')[:3, :3]
        # Apply rotation
        pc_tf = np.inner(pc_arr, rot)
        # Combine rotation and translation
        pc_tf += translation
        return pc_tf

    def ramp_detection(
            self, plane, g_vec, n_vec, min_angle, max_angle, 
            min_height, max_height, min_width, max_width, 
            min_length, max_length, logging=False):
        """Checks if conditions to be considered a ramp are fullfilled."""
        # Convert pcl plane to numpy array
        plane_array = np.array(plane)
        # Calculate angle [deg] between new and previously recorded normal vector of ground
        angle = self.angle_calc(g_vec, n_vec)
        # Get ramp height (Difference between z-values of furthest and nearest point)
        height = max(plane_array[:, 2]) - min(plane_array[:, 2])
        # Get ramp width (Difference between y-values)
        width = max(plane_array[:, 1]) - min(plane_array[:, 1])
        # Length in x direction 
        x = max(plane_array[:, 0]) - min(plane_array[:, 0])
        # Get ramp length (using pythagorean theorem)
        length = math.sqrt(x**2 + height**2)
        # Ramp distance (x-value of nearest point of the plane)
        dist = min(plane_array[:,0])
        data = [angle, dist, width, length, height]
        if min_angle <= angle <= max_angle:
            pass
        else:
            return False, data
        if min_height <= height <= max_height:
            pass
        else:
            return False, data
        if min_width <= width <= max_width:
            pass
        else:
            return False, data
        if min_length <= length <= max_length:
            pass
        else:
            return False, data
        return True, data

    def ransac(self, pc):
        """Finds inliers and normal vector of dominant plane"""
        # 50?
        seg = pc.make_segmenter_normals(self.ransac_make_segmenter_normals)
        # Doubles the speed if True
        seg.set_optimize_coefficients(True)
        seg.set_model_type(pcl.SACMODEL_NORMAL_PLANE)
        seg.set_method_type(pcl.SAC_RANSAC)
        # How close a point must be to model to be considered inlier
        seg.set_distance_threshold(self.distance_threshold)
        # normal_distance_weight?
        seg.set_normal_distance_weight(self.distance_weight)
        # How many tries
        seg.set_max_iterations(100)
        indices, coefficients = seg.segment()
        return indices, coefficients

    def split_pc(self, pc, inliers):
        """Extract detected plane from point cloud and split into two pcs"""
        # Get point cooridnates of plane
        detected_plane = [pc[i] for i in inliers]
        # Point cloud of detected plane (inliers)
        pc_inliers = pc.extract(inliers)
        # Point cloud of outliers
        outlier_indices = list(set(np.arange(pc.size)).symmetric_difference(inliers))
        pc_outliers = pc.extract(outlier_indices)
        return pc_outliers, detected_plane, pc_inliers

    def is_plane_near_ground(self, v, threshold=0.8):
        """Returns True if plane is on the ground (and false if e.g. side wall)"""
        return abs(v[2]) > threshold   

    def angle_calc(self, v1, v2, degrees=True):
        """Calculate angle between two vectors (planes)"""
        # Assuming both vectors can be rotated alongside one axis to be aligned
        dot = np.dot(v1, v2)
        if dot <= 1:
            angle = np.arccos(dot)
        else:
            angle = 0
        if degrees is True:
            return np.degrees(angle)
        else:
            return angle

In [ ]:
def dumb_optimizer(varIndex, start, end, steps=6, visualize=True):
    # Variables that can be changed
    var_names = ['ransac_distance_thresh', 'ransac_weight', 'ransac_make_seg',
        'voxel_leaf', 'plane_near_ground', 'max_iter', 'min_points', 'cut']
    # Standard parameters
    params = [0.017, 0.125, 50, 0.1, 0.8, 4, 100, [0, 30, -3.5, 3.5, -1, 1.5]]
    # List for visualization
    vis_list = []
    # Change parameter value every run
    for var_val in np.linspace(start, end, steps):
        # Adjust input parameter
        params[varIndex] = var_val
        # Create instance of class
        vd = VisualDetection(ransac_distance_thresh=params[0], ransac_weight=params[1], 
        ransac_make_seg=params[2], voxel_leaf=params[3], plane_near_ground=params[4], 
        max_iter=params[5], min_points=params[6], cut=params[7])
        # Lists to fill, will contain entry for each plane
        planes = []
        ramp_stats = []
        # Timestop
        start_t = time.time()
        for i in range(len(lidar)):
            plane_points, data = vd.spin(lidar[i], pose[i])
            planes.append(plane_points)
            ramp_stats.append(data)
        # Timestop end
        runtime = time.time() - start_t

        # Remove empty lists (when no ramp has been detected)
        ramp_arrays = [x for x in planes if x != []]
        ramp_stats = [x for x in ramp_stats if x != []]

        # Convert list to dictionary
        dic = []
        for i, arr in enumerate(ramp_arrays):
            for j, point in enumerate(arr):
                dic.append(
                    {
                        'sampleIdx': i,
                        'pointIdx': j,
                        'x': point[0],
                        'y': point[1],
                        'z': point[2]
                    }
                )
        # And finally to pandas Dataframe
        df = pd.DataFrame(dic)
        # Adjust column order
        df = df[['sampleIdx', 'pointIdx', 'x', 'y', 'z']]

        # Ground truth coordinates of ramp (measured by using globalmap points)
        x_range = [-21, -9]
        y_range = [35, 39] 

        # Check if a point lies within ramp region
        lies_inside = []
        for i,x in enumerate(df['x']):
            if x_range[0] < x < x_range[1]:
                if y_range[0] < df['y'][i] < y_range[1]:
                    # True if x and y coordinate inside region
                    lies_inside.append(True)
                else:
                    lies_inside.append(False)
            else:
                lies_inside.append(False)
        # Add column (if point lies in region) to data frame
        df['inlier'] = lies_inside

        # Calculate how many points of each sample lie in ramp region
        true_inliers = []
        samples_num = df.sampleIdx.max() + 1
        for i in range(samples_num):
            # Bool list of inliers and outliers of sample
            bool_lst = df[df['sampleIdx'] == i]['inlier']
            # Percentage of inliers of sample
            true_inliers.append(sum(bool_lst) / float(len(bool_lst)))

        # Average percentage of points in ramp region per sample
        score = np.array(true_inliers).mean()
        print('With parameter {} = {} the score was {:05.2f}% with a runtime of {:05.2f}s'.format(
            var_names[varIndex], var_val, score*100, runtime))
        # Todo: Add expected value of ramp (sth like time from when ramp is visible
        # Todo: until not times the rate)
        print('Out of ... expected samples, {} were detected.'.format(len(true_inliers)))
        vis_list.append([var_names[varIndex], var_val, score*100, runtime])
    if visualize:
        df = pd.DataFrame(vis_list)
        fig = make_subplots(rows=1, cols=2, 
        subplot_titles=('Score (higher is better)', 'Runtime (lower is better)'),
        y_title='Parameter value')
        fig.add_trace(go.Bar(
            y = df.iloc[:,1], x=df.iloc[:,2],
            name="Score",
            orientation='h', 
            textposition='inside'
        ))
        fig.add_trace(go.Bar(
            y = df.iloc[:,1], x=df.iloc[:,3],
            name="Runtime",
            orientation='h', 
            marker_color='orange',
            textposition='inside'
        ), row=1, col=2)
        fig.update_traces(texttemplate='%{y:.3f}', insidetextanchor="middle")
        fig.update_xaxes(title_text="Percentage [%]", row=1, col=1)
        fig.update_xaxes(title_text="Time [s]", row=1, col=2)
        fig.update_layout(title_text="Performance for different " + vis_list[0][0] + " values")
        fig.show()
        return fig

The following parameters can be tested (standard value on right hand side):

0. ransac_distance_thresh=0.017
1. ransac_weight=0.125
2. ransac_make_seg=50,
3. voxel_leaf=0.1
4. plane_near_ground=0.8
5. cut=[0, 30, -3.5, 3.5, -1, 1.5]
6. max_iter=4
7. min_points=100

The optimizer takes the following input:

`def dumb_optimizer(varIndex, start, end, steps=6)`

The score and runtime are displayed in the following plots.

The score is calculated as the average value of inlier percentage per detected plane, that is considered a ramp. Hence it is also possible to get a perfect score of 100%, while only one plane was detected. To tackle this problem, another plot was created further down, which shows the ratio and count of true positives and false positives.

The best way to evaluate would be to set a constant number of positives manually. E.g. if the car is 20m in front of a ramp it should be able to detect it, until the lower threshold has been reached, e.g. 5m before the ramp, at which the lidar can not distinguish between ground and ramp anymore.

Because right now the amount of positives changes for each sample / parameter change, which is not good.

In [5]:
# Standard parameters
params = [0.017, 0.125, 50, 0.1, 0.8, 4, 100, [0, 30, -3.5, 3.5, -1, 1.5]]
# Change each parameter by 50 to 100% and explore their influence
for i in range(7):
    if i != 4:
        dumb_optimizer(i, params[i]*0.5, params[i]*2, 4)

With parameter ransac_weight = 0.125 the score was 81.05% with a runtime of 12.66s
Out of ... expected samples, 69 were detected.


Now visualize the number of true positives and false positives instead of the score

In [ ]:
def dumb_optimizer2(varIndex, start, end, steps=6, visualize=True):
    # Variables that can be changed
    var_names = ['ransac_distance_thresh', 'ransac_weight', 'ransac_make_seg',
        'voxel_leaf', 'plane_near_ground', 'max_iter', 'min_points', 'cut']
    # Standard parameters
    params = [0.017, 0.125, 50, 0.1, 0.8, 4, 100, [0, 30, -3.5, 3.5, -1, 1.5]]
    # List for visualization
    vis_list = []
    # Change parameter value every run
    for var_val in np.linspace(start, end, steps):
        # Adjust input parameter
        params[varIndex] = var_val
        # Create instance of class
        vd = VisualDetection(ransac_distance_thresh=params[0], ransac_weight=params[1], 
        ransac_make_seg=params[2], voxel_leaf=params[3], plane_near_ground=params[4], 
        max_iter=params[5], min_points=params[6], cut=params[7])
        # Lists to fill, will contain entry for each plane
        planes = []
        ramp_stats = []
        # Timestop
        start_t = time.time()
        for i in range(len(lidar)):
            plane_points, data = vd.spin(lidar[i], pose[i])
            planes.append(plane_points)
            ramp_stats.append(data)
        # Timestop end
        runtime = time.time() - start_t

        # Remove empty lists (when no ramp has been detected)
        ramp_arrays = [x for x in planes if x != []]
        ramp_stats = [x for x in ramp_stats if x != []]

        # Convert list to dictionary
        dic = []
        for i, arr in enumerate(ramp_arrays):
            for j, point in enumerate(arr):
                dic.append(
                    {
                        'sampleIdx': i,
                        'pointIdx': j,
                        'x': point[0],
                        'y': point[1],
                        'z': point[2]
                    }
                )
        # And finally to pandas Dataframe
        df = pd.DataFrame(dic)
        # Adjust column order
        df = df[['sampleIdx', 'pointIdx', 'x', 'y', 'z']]

        # Ground truth coordinates of ramp (measured by using globalmap points)
        x_range = [-21, -9]
        y_range = [35, 39] 

        # Check if a point lies within ramp region
        lies_inside = []
        for i,x in enumerate(df['x']):
            if x_range[0] < x < x_range[1]:
                if y_range[0] < df['y'][i] < y_range[1]:
                    # True if x and y coordinate inside region
                    lies_inside.append(True)
                else:
                    lies_inside.append(False)
            else:
                lies_inside.append(False)
        # Add column (if point lies in region) to data frame
        df['inlier'] = lies_inside

        # Calculate how many points of each sample lie in ramp region
        true_inliers = []
        samples_num = df.sampleIdx.max() + 1
        for i in range(samples_num):
            # Bool list of inliers and outliers of sample
            bool_lst = df[df['sampleIdx'] == i]['inlier']
            # Percentage of inliers of sample
            true_inliers.append(sum(bool_lst) / float(len(bool_lst)))

        # Average percentage of points in ramp region per sample
        score = np.array(true_inliers).mean()
        print('With parameter {} = {} the score was {:05.2f}% with a runtime of {:05.2f}s'.format(
            var_names[varIndex], var_val, score*100, runtime))
        # Todo: Add expected value of ramp (sth like time from when ramp is visible
        # Todo: until not times the rate)
        print('Out of ... expected samples, {} were detected.'.format(len(true_inliers)))
        vis_list.append([var_names[varIndex], var_val, score*100, runtime, true_inliers])
    if visualize:
        df = pd.DataFrame(vis_list)
        tps = []
        fps = []
        for sample in df.iloc[:,4]:
            tp = sum(np.array(sample) > 0.5)
            tps.append(tp)
            fps.append(len(sample) - tp)
        df[5] = tps
        df[6] = fps
        fig = make_subplots(rows=1, cols=2, 
        subplot_titles=('TruePositives vs FalsePositives\n(Should be high tp and low fp))', 'Runtime (lower is better)'),
        y_title='Parameter value')
        fig.add_trace(go.Bar(
            y = df.iloc[:,1], x=df.iloc[:,5],
            name="True positives",
            orientation='h', 
            textposition='inside', 
            offsetgroup=0
        ))
        fig.add_trace(go.Bar(
            y = df.iloc[:,1], x=df.iloc[:,6],
            name="False positives",
            orientation='h', 
            textposition='inside', 
            offsetgroup=0
        ))
        fig.add_trace(go.Bar(
            y = df.iloc[:,1], x=df.iloc[:,3],
            name="Runtime",
            orientation='h', 
            marker_color='orange',
            textposition='inside'
        ), row=1, col=2)
        fig.update_traces(texttemplate='%{y:.3f}', insidetextanchor="middle")
        fig.update_xaxes(title_text="Sample count", row=1, col=1)
        fig.update_xaxes(title_text="Time [s]", row=1, col=2)
        fig.update_layout(title_text="Performance for different " + vis_list[0][0] + " values")
        fig.show()
        return fig

In [ ]:
# Standard parameters
params = [0.017, 0.125, 50, 0.1, 0.8, 4, 100, [0, 30, -3.5, 3.5, -1, 1.5]]
# Change each parameter by 50 to 100% and explore their influence
for i in range(7):
    if i != 4:
        dumb_optimizer2(i, params[i]*0.5, params[i]*2, 4)